# LLM Example: Query documents 

This example takes a (german) law description and allows you to query it.

> Looking for someone to kickstart your Generative AI project? \
  Write me a message (jannis.gansen@codecamp-n.com) and check out https://www.codecamp-n.com/  \
  ![CodeCampN.png](../logo.png)

## 1. ⛓️ Install dependencies

- We'll use langchain for creating our agent
- openai for accessing the OpenAI models
- pypdf for parsing PDF files
- tiktoken as we need to create embeddings
- chromadb for creating a local vector store


In [ ]:
!pip install langchain openai pypdf tiktoken chromadb

## 2. 🔑 Setup credentials

You have two options to set your credentials:



### Option 1: Set environment directly in the notebook

Insert your OpenAPI Key into the OPENAI_API_KEY environment variable, but make sure to not share this information:

In [39]:
import os

os.environ["OPENAI_API_KEY"] = "** Your Key **"

### Option 2: Query via getpass

This requries the user of the notebook to provide the key.

Pro: You can't lose your key \
Con: You need to enter it everytime you restart the kernel.

In [ ]:
from getpass import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass('Your OPENAI key: ')

## 3. Load and preprocess PDF
Langchain provides a PyPDFLoader implementation that simplifies
loading and splittng PDF documents by pages.


### 3.1 Download a doument

First lets download the german "Umsatzsteuergesetz" and use this as our knowledge source.

In [3]:
from urllib import request

remote_url = 'https://www.gesetze-im-internet.de/ustg_1980/UStG.pdf'

local_file = 'ustg_2023.pdf'
request.urlretrieve(remote_url, local_file)
doc_name = local_file


>  (Optional) If you want to try with your own document you can upload it here in a colab environment:

In [ ]:
from google.colab import files
uploaded = files.upload()

doc_name = uploaded.keys()[0]

### 3.2 Create embeddings and store them in a vector database

We'll use chroma as a vector database and store the pdf. For this we'll split the document first using langchains [RecursiveTextSplitter](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter), as this tries to keep paragraphs together.

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader(doc_name)
pages = loader.load()
text_splitter = RecursiveCharacterTextSplitter(

    chunk_size = 4096,
    chunk_overlap  = 128,
    length_function = len,
    is_separator_regex = False,
)
docs = text_splitter.split_documents(pages)
len(docs)

Next we'll create embeddings for the document and add them to our vector store:

In [5]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(docs, embeddings, persist_directory="./chroma_db_ustg")




Let's test whether we find information in our vector store. We'll use a max marginal relevance search, as this produces good results in our demo case.

In [ ]:
query = "Kleinunternehmer"
found_docs = db.max_marginal_relevance_search(query, k=2, fetch_k=10)

found_docs

## 4. ⛓️ Use the vector store as a retriever

We have everything in place to build our agent now. Our Agent will get the document search as a retriever and use a ZERO SHOT REACT DESCRIPTION type. Feel free to experiment with different settings!

In [15]:
from langchain.chains import RetrievalQA
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

# or use gpt-4 if it's available in your account, as it performs better
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

ustgSearch = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=db.as_retriever(search_type="mmr")
)
tools = [
    Tool(
        name="UstG Suche",
        func=ustgSearch.run,
        description="Nützlich wenn du fragen zu gesetzliche Rahmenbedingungen aus dem UstG hast (Umsatzsteuergesetz)",
    ),
]
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)


In [17]:
# @title UstG Prompt
# @markdown ...and finally you can test your prompt!
prompt = "Wann ist man Kleinunternehmer laut UstG? " # @param {type:"string"}

agent.run(prompt)



> Entering new AgentExecutor chain...
Um diese Frage zu beantworten, muss ich das Umsatzsteuergesetz (UstG) konsultieren, um die genauen Bedingungen für die Einstufung als Kleinunternehmer zu finden.
Action: UstG Suche
Action Input: Kleinunternehmer
Observation: Kleinunternehmer sind Unternehmer, die im Inland oder in den in § 1 Abs. 3 bezeichneten Gebieten ansässig sind und deren Umsatz im vorangegangenen Kalenderjahr 22.000 Euro nicht überstiegen hat und im laufenden Kalenderjahr 50.000 Euro voraussichtlich nicht übersteigen wird. Für diese Unternehmer wird die Umsatzsteuer für Umsätze im Sinne des § 1 Abs. 1 Nr. 1 nicht erhoben.
Thought:Ich habe jetzt die Informationen, um die Frage zu beantworten.
Final Answer: Laut Umsatzsteuergesetz (UstG) ist man ein Kleinunternehmer, wenn man im Inland oder in den in § 1 Abs. 3 bezeichneten Gebieten ansässig ist und der Umsatz im vorangegangenen Kalenderjahr 22.000 Euro nicht überstiegen hat und im laufenden Kalenderjahr 50.000 Euro voraussic

'Laut Umsatzsteuergesetz (UstG) ist man ein Kleinunternehmer, wenn man im Inland oder in den in § 1 Abs. 3 bezeichneten Gebieten ansässig ist und der Umsatz im vorangegangenen Kalenderjahr 22.000 Euro nicht überstiegen hat und im laufenden Kalenderjahr 50.000 Euro voraussichtlich nicht übersteigen wird. Für diese Unternehmer wird die Umsatzsteuer für Umsätze im Sinne des § 1 Abs. 1 Nr. 1 nicht erhoben.'

## 🚀 Where do we go from here?

### Improve the prompt

You can try to embed relevant inforamtion directly using a Prompt Template
